In [5]:
import numpy as np
import pandas as pd

In [61]:
!pip install tensorflow

  Using cached tensorflow-2.6.0-cp38-cp38-win_amd64.whl (423.2 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.41.0-cp38-cp38-win_amd64.whl (3.2 MB)
  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
  Using cached h5py-3.1.0-cp38-cp38-win_amd64.whl (2.7 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached clang-5.0-py3-none-any.whl
  Using cached absl_py-0.15.0-py3-none-any.whl (132 kB)
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorflow_estimator-2.6.0-py2.py3-none-any.whl (462 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached Markdown-3.3.4-py3-none-any.whl (97 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl 

In [3]:
train_dir = "Dataset"

In [4]:
!pip install opencv-python

In [4]:
import os
import cv2
import random 
from tqdm import tqdm

In [6]:
os.listdir(train_dir)

['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

In [15]:
Class = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia'] 

In [8]:
len(os.listdir(train_dir+"/COVID"))

3575

In [9]:
len(os.listdir(train_dir+"/Lung_Opacity"))

6012

In [10]:
len(os.listdir(train_dir+"/Normal"))

6300

In [12]:
len(os.listdir(train_dir+"/Viral Pneumonia"))

5151

In [13]:
img_size = 100

In [14]:
def create_data(dir_name):
    dataset =[]
    for cl in Class:
        cl_path = os.path.join(dir_name,cl)
        class_num = Class.index(cl)
        for img in os.listdir(cl_path):
            try:
                img_arr = cv2.imread(os.path.join(cl_path,img))
                img_arr=cv2.resize(img_arr,(img_size,img_size))
                img_arr = cv2.cvtColor(img_arr,cv2.COLOR_BGR2GRAY)
                dataset.append([img_arr,class_num])
            except:
                pass
    return dataset

In [15]:
dataset = create_data(train_dir)

In [16]:
random.shuffle(dataset)

In [17]:
len(dataset)

21038

In [18]:
X =[]
y=[]

for features,label in dataset:
    X.append(features)
    y.append(label)

In [19]:
np.array(X).shape

(21038, 100, 100)

In [20]:
X =np.array(X).reshape(-1,img_size,img_size,1)

In [21]:
X.shape

(21038, 100, 100, 1)

In [22]:
y = np.array(y)

In [23]:
X =X/255.0

In [24]:
from keras.utils import np_utils
y = np_utils.to_categorical(y,4)

## Apply CNN

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Conv2D , MaxPooling2D ,Flatten

In [26]:
model = Sequential()

model.add(Conv2D(64 , (3,3) ,input_shape = X.shape[1:] ,activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

model.add(Conv2D(128 , (3,3) ,padding = "same", activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

model.add(Conv2D(96 , (3,3) ,padding = "same", activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

model.add(Flatten())

model.add(Dense(activation='relu' , units =128 ))

model.add(Dense(4,activation = 'softmax'))

model.compile(loss="categorical_crossentropy" , optimizer= 'adam' ,metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 49, 49, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 96)        110688    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 96)        0         
_________________________________________________________________
flatten (Flatten)            (None, 13824)             0

In [28]:
model_result = model.fit(X, y, validation_split = 0.2, epochs =15 ,batch_size=32)


Epoch 1/15
526/526 [==============================] - 628s 1s/step - loss: 0.5586 - accuracy: 0.7796 - val_loss: 0.3837 - val_accuracy: 0.8543
Epoch 2/15
526/526 [==============================] - 498s 947ms/step - loss: 0.3482 - accuracy: 0.8705 - val_loss: 0.3183 - val_accuracy: 0.8757
Epoch 3/15
526/526 [==============================] - 414s 788ms/step - loss: 0.2747 - accuracy: 0.8948 - val_loss: 0.3033 - val_accuracy: 0.8824
Epoch 4/15
526/526 [==============================] - 405s 771ms/step - loss: 0.2350 - accuracy: 0.9103 - val_loss: 0.2835 - val_accuracy: 0.8912
Epoch 5/15
526/526 [==============================] - 404s 768ms/step - loss: 0.1849 - accuracy: 0.9287 - val_loss: 0.2775 - val_accuracy: 0.8997
Epoch 6/15
526/526 [==============================] - 61252s 117s/step - loss: 0.1580 - accuracy: 0.9390 - val_loss: 0.2818 - val_accuracy: 0.8985
Epoch 7/15
526/526 [==============================] - 623s 1s/step - loss: 0.1276 - accuracy: 0.9525 - val_loss: 0.2740 - val_

In [29]:
model.save("cnn_model.h5")

### test our model

In [1]:
from tensorflow.keras.models import load_model 

In [2]:
model = load_model("cnn_model.h5")

In [7]:
pic2_array = cv2.imread("Dataset/COVID/COVID-1035.png", cv2.IMREAD_GRAYSCALE)
new_array2 = cv2.resize(pic2_array,(100,100))

img2_batch = np.expand_dims(new_array2, axis=0)
uimg2 = np.resize(img2_batch,(1,100,100,1))

uimg2 = uimg2/255.0
pred2 = model.predict(uimg2)

pred2

array([[9.9503505e-01, 4.9431310e-03, 9.3672406e-06, 1.2477434e-05]],
      dtype=float32)

In [16]:
(np.round(pred2,4))*100

array([[99.5 ,  0.49,  0.  ,  0.  ]], dtype=float32)

### tensorflow.js

In [1]:
from tensorflow.keras.models import load_model 

In [2]:
model = load_model("cnn_model.h5")

In [4]:
pip install tensorflowjs

     |████████████████████████████████| 64 kB 415 kB/s eta 0:00:01
     |████████████████████████████████| 108 kB 685 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflowjs as tfjs


In [6]:
tfjs.converters.save_keras_model(model, "xray_tfjs_model")

In [ ]:
#prediction instructions :
1) images should be read in GrayScale
2) 100*100
3)expand_dims

In [ ]:
testing...............

Class = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia'] 

In [24]:
pic2_array = cv2.imread("imgs/p2.jpeg", cv2.IMREAD_GRAYSCALE)
new_array2 = cv2.resize(pic2_array,(100,100))
img2_batch = np.expand_dims(new_array2, axis=0)
uimg2 = np.resize(img2_batch,(1,100,100,1))

uimg2 = uimg2/255.0
pred2 = model.predict(uimg2)

pred2
print((np.round(pred2,19))*1000000000000000000000)

[[0.0 6899.999871504985 0.0 1e+21]]


In [10]:
pred2[0].argmax()

2